In [2]:
import pandas as pd
import nltk

In [3]:
from joblib import dump, load

In [4]:
data = pd.read_csv('help_title_v2.csv')

In [5]:
data.head()

,section_1,url_1,section_2,url_2,section_3,url_3,section_4,url_4,section_text,keywords,text_len
0,"Tasks, Folders, Projects and Spaces",https://help.wrike.com/hc/en-us/categories/200...,Tasks,https://help.wrike.com/hc/en-us/sections/20189...,Tasks,https://help.wrike.com/hc/en-us/articles/20960...,Create a Task,https://help.wrike.com/hc/en-us/articles/20960...,Create a Task\n\nSelect a Folder or Project fr...,[],440.0
1,"Tasks, Folders, Projects and Spaces",https://help.wrike.com/hc/en-us/categories/200...,Tasks,https://help.wrike.com/hc/en-us/sections/20189...,Tasks,https://help.wrike.com/hc/en-us/articles/20960...,Assign a Task,https://help.wrike.com/hc/en-us/articles/20960...,Assign a Task\nChoose the names of people to w...,"['invite a user to Wrike', 'email notification...",1128.0
2,"Tasks, Folders, Projects and Spaces",https://help.wrike.com/hc/en-us/categories/200...,Tasks,https://help.wrike.com/hc/en-us/sections/20189...,Tasks,https://help.wrike.com/hc/en-us/articles/20960...,Schedule a Task,https://help.wrike.com/hc/en-us/articles/20960...,"Schedule a Task\nIn the List view, you can set...","['Wrike for Professional Services', 'Wrike Res...",1131.0
3,"Tasks, Folders, Projects and Spaces",https://help.wrike.com/hc/en-us/categories/200...,Tasks,https://help.wrike.com/hc/en-us/sections/20189...,Tasks,https://help.wrike.com/hc/en-us/articles/20960...,Tag a Task (Organize Tasks into Folders),https://help.wrike.com/hc/en-us/articles/20960...,Tag a Task (Organize Tasks into Folders)\nWhen...,['mass actions'],1008.0
4,"Tasks, Folders, Projects and Spaces",https://help.wrike.com/hc/en-us/categories/200...,Tasks,https://help.wrike.com/hc/en-us/sections/20189...,Tasks,https://help.wrike.com/hc/en-us/articles/20960...,Follow a Task,https://help.wrike.com/hc/en-us/articles/20960...,Follow a Task\nFollowing a task is a great way...,"['Activity Stream', 'instant email notificatio...",1040.0


In [6]:
data['section_1'].unique()

array(['Tasks, Folders, Projects and Spaces',
       'Types of Accounts and Licenses', 'Work Views', 'Monitoring Panel',
       'Integrations', 'Account Management', 'Security', 'Apps',
       'Getting Started'], dtype=object)

In [7]:
data['section_2'].unique()

array(['Tasks', 'Folders and Projects', 'Spaces',
       'Proofing and Approvals', 'Communication', 'Advanced',
       'Organization', 'Accounts', 'Licenses', 'Gantt Chart',
       'Table View', 'Timelog View', 'All the Other Views',
       'Wrike Analyze', 'Wrike Resource', 'Calendars', 'Reporting',
       'More Monitoring Panel Views', 'Automated Monitoring',
       'Wrike Integrate', 'Email', 'Add-ins and Extensions',
       'Import and Export', 'Wrike, GitHub, and JIRA Syncs',
       'Everything Else', 'Troubleshooting', 'General Account Management',
       'Personal User License', 'Billing', 'Single Sign-On',
       'More Security Features', 'Desktop', 'Mobile',
       'Manager Guide (for account owners and admins)',
       'Team Member Guide (for account users)'], dtype=object)

In [8]:
data['section_3'].unique()

array(['Tasks', 'Task View', 'Subtasks', 'Subtask Alignment ',
       'Attachments', 'Milestones', 'Backlogged Tasks',
       'List View - Sorting and Prioritizing Tasks', 'Mass Editing',
       'Sharing Tasks', 'Duplicate Tasks and Subtasks', 'Star a Task',
       'Importance', 'Task Status', 'Request a Status Update',
       'Make a Task Recurrent', 'Delete Tasks', 'Print Tasks',
       'Create, Color Code, and Edit Folders and Projects',
       'Folder and Project Info Panels', 'Projects',
       'Project Wizard and Default View Picker',
       'Subfolders and Subprojects', 'Sharing Folders and Projects',
       'Access Roles', 'Duplicate a Folder or Project',
       'Delete a Folder or Project', 'Project Progress',
       'Spaces Overview', 'Personal Space',
       'Manage Spaces (for Space Admins)', 'Bookmarks',
       'Wrike Proof for HTML', 'Approvals', 'Proofing', 'File Approvals',
       'Guest Reviews', 'Review and Approve Files (for Guest Users)',
       'Wrike Publish: Medi

In [9]:
information_section_list = ['Wrike and Bitbucket Sync: Overview',
       'Wrike and GitLab Sync: Overview',
       'Wrike and GitHub Sync: Overview',
       'Wrike and JIRA Sync: Overview',
       'Wrike and JIRA Sync: Setup Guide','Step 1: Learning the Fundamentals',
       'Step 2: Setting Up Your Workspace',
       'Step 3: Team Onboarding and Collaboration',
       'Step 4: Monitoring Work Progress',
       'Step 5: Tailoring Wrike to Your Team’s Needs',
       'Step 6. Making Remote Work Productive', 'Step 1: Getting Settled',
       'Step 2: Launching New Projects', 'Step 3: Collaborating in Wrike',
       'Step 4: Work from Home Productively']

In [10]:
data = data[~data['section_3'].isin(information_section_list)]

In [11]:
data.shape

(1196, 11)

In [12]:
data = data[data['section_text'] != '']

In [13]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [14]:
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")

In [15]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [16]:
#Preprocess function
def preprocess_text(text, tokenizer=RegexpTokenizer(r'\w+'), 
                    lemmatizer=WordNetLemmatizer(), stopwords=stopwords.words("english")):
    tokens = tokenizer.tokenize(text)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if token not in english_stopwords\
              and token != " " ]
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    clean_text = " ".join(lemmas)
    return clean_text

In [17]:
data.head()

,section_1,url_1,section_2,url_2,section_3,url_3,section_4,url_4,section_text,keywords,text_len
0,"Tasks, Folders, Projects and Spaces",https://help.wrike.com/hc/en-us/categories/200...,Tasks,https://help.wrike.com/hc/en-us/sections/20189...,Tasks,https://help.wrike.com/hc/en-us/articles/20960...,Create a Task,https://help.wrike.com/hc/en-us/articles/20960...,Create a Task\n\nSelect a Folder or Project fr...,[],440.0
1,"Tasks, Folders, Projects and Spaces",https://help.wrike.com/hc/en-us/categories/200...,Tasks,https://help.wrike.com/hc/en-us/sections/20189...,Tasks,https://help.wrike.com/hc/en-us/articles/20960...,Assign a Task,https://help.wrike.com/hc/en-us/articles/20960...,Assign a Task\nChoose the names of people to w...,"['invite a user to Wrike', 'email notification...",1128.0
2,"Tasks, Folders, Projects and Spaces",https://help.wrike.com/hc/en-us/categories/200...,Tasks,https://help.wrike.com/hc/en-us/sections/20189...,Tasks,https://help.wrike.com/hc/en-us/articles/20960...,Schedule a Task,https://help.wrike.com/hc/en-us/articles/20960...,"Schedule a Task\nIn the List view, you can set...","['Wrike for Professional Services', 'Wrike Res...",1131.0
3,"Tasks, Folders, Projects and Spaces",https://help.wrike.com/hc/en-us/categories/200...,Tasks,https://help.wrike.com/hc/en-us/sections/20189...,Tasks,https://help.wrike.com/hc/en-us/articles/20960...,Tag a Task (Organize Tasks into Folders),https://help.wrike.com/hc/en-us/articles/20960...,Tag a Task (Organize Tasks into Folders)\nWhen...,['mass actions'],1008.0
4,"Tasks, Folders, Projects and Spaces",https://help.wrike.com/hc/en-us/categories/200...,Tasks,https://help.wrike.com/hc/en-us/sections/20189...,Tasks,https://help.wrike.com/hc/en-us/articles/20960...,Follow a Task,https://help.wrike.com/hc/en-us/articles/20960...,Follow a Task\nFollowing a task is a great way...,"['Activity Stream', 'instant email notificatio...",1040.0


In [18]:
data['section_text_cleaned'] = data['section_text'].apply(lambda x: preprocess_text(str(x)))

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
TF_X = vectorizer.fit_transform(data['section_text_cleaned'])

## Dumping vectorizer

In [20]:
dump(vectorizer, 'vectorizer.joblib')

['vectorizer.joblib']

In [21]:
from sklearn.preprocessing import OneHotEncoder

section_1_encoder = OneHotEncoder(handle_unknown='ignore')
section_2_encoder = OneHotEncoder(handle_unknown='ignore')
section_3_encoder = OneHotEncoder(handle_unknown='ignore')

section_1_ohe = section_1_encoder.fit_transform(data['section_1'].values.reshape(-1, 1))
section_2_ohe = section_2_encoder.fit_transform(data['section_2'].values.reshape(-1, 1))
section_3_ohe = section_3_encoder.fit_transform(data['section_3'].values.reshape(-1, 1))

## Dumping encoders

In [22]:
dump(section_1_encoder, 'section_1_encoder.joblib')
dump(section_2_encoder, 'section_2_encoder.joblib')
dump(section_3_encoder, 'section_3_encoder.joblib')

['section_3_encoder.joblib']

In [23]:
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.multioutput import ClassifierChain
from sklearn.multiclass import OneVsRestClassifier

In [24]:
def predict_one_class(train_data, submit_data, y, model, n_splits=3, silent=False):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    to_check = pd.DataFrame(train_data.copy())
    to_check[f'predict_proba'] = 0
    pred = np.zeros(len(submit_data))
    y_arr = y
    for i, (train_index, valid_index) in enumerate(skf.split(train_data, y_arr)):        
        X_train = train_data[train_index]
        X_valid = train_data[valid_index]
        y_train = y_arr[train_index]
        y_valid = y_arr[valid_index] 
        
        model.fit(X_train, y_train)
        
        prediction_skf = model.predict_proba(submit_data)[:,1]
        pred += prediction_skf / skf.n_splits  
        
        prediction = model.predict_proba(X_valid)[:,1]
        to_check.loc[valid_index, f'predict_proba'] = prediction
                
    local_auc = roc_auc_score(y, to_check[f'predict_proba'])
    
    if not silent: 
        print(f"ROC_AUC_local: {local_auc}")
    
    return pred,to_check[f'predict_proba'],local_auc

In [25]:
import numpy as np

In [26]:
Y = section_1_ohe.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(TF_X.toarray(), Y, test_size=0.2)

cv_scores = []
probas = []
cross_vals = []

columns = Y.shape[1]
for i in range(columns):
    y_train = Y_train[:, i]
    clf = LogisticRegression()
    proba, cross_val, cv_score = predict_one_class(X_train, X_test, y_train, clf, n_splits=3)  
    probas.append(proba)
    cross_vals.append(cross_val)
    cv_scores.append(cv_score)
print ()
print(f'TOTAL CV:{np.mean(cv_scores)}')

ROC_AUC_local: 0.9480401457235558
ROC_AUC_local: 0.9073200992555832
ROC_AUC_local: 0.9687069164931761
ROC_AUC_local: 0.9704847473160514
ROC_AUC_local: 0.9714160346016424
ROC_AUC_local: 0.9234623981541568
ROC_AUC_local: 0.9738904029589254
ROC_AUC_local: 0.9179223744292239

TOTAL CV:0.9476553898665394


In [27]:
import warnings
warnings.filterwarnings("ignore")

In [28]:
Y = section_2_ohe.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(TF_X.toarray(), Y, test_size=0.2)

cv_scores = []
probas = []
cross_vals = []

columns = Y.shape[1]
for i in range(columns):
    y_train = Y_train[:, i]
    clf = LogisticRegression()
    try:
        proba, cross_val, cv_score = predict_one_class(X_train, X_test, y_train, clf, n_splits=3)
    except:
        pass
    probas.append(proba)
    cross_vals.append(cross_val)
    cv_scores.append(cv_score)
print ()
print(f'TOTAL CV:{np.mean(cv_scores)}')

ROC_AUC_local: 0.9334398296059638
ROC_AUC_local: 0.9607702427939241
ROC_AUC_local: 0.9561837006923712
ROC_AUC_local: 0.9530611251703328
ROC_AUC_local: 0.6666666666666667
ROC_AUC_local: 0.9311275372521413
ROC_AUC_local: 0.9411764705882352
ROC_AUC_local: 0.9680353200883003
ROC_AUC_local: 0.9821052631578948
ROC_AUC_local: 0.9801743679163034
ROC_AUC_local: 0.9767961650442608
ROC_AUC_local: 0.9515141347202416
ROC_AUC_local: 0.9554221379584767
ROC_AUC_local: 0.9762353250394252
ROC_AUC_local: 0.9990691489361703
ROC_AUC_local: 0.9932259521300617
ROC_AUC_local: 0.9466662921979441
ROC_AUC_local: 0.9269515281292583
ROC_AUC_local: 0.9786593614718615
ROC_AUC_local: 0.8447973438576708
ROC_AUC_local: 0.9239281400966184
ROC_AUC_local: 0.9280948067632849
ROC_AUC_local: 0.979021724643476
ROC_AUC_local: 0.9539127895658944
ROC_AUC_local: 0.992495381773399
ROC_AUC_local: 0.942508506394462
ROC_AUC_local: 0.9282516172965611
ROC_AUC_local: 0.9892706131078225
ROC_AUC_local: 0.9892626231527093
ROC_AUC_local: 0.

In [29]:
Y = section_3_ohe.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(TF_X.toarray(), Y, test_size=0.2)

cv_scores = []
probas = []
cross_vals = []

columns = Y.shape[1]
for i in range(columns):
    y_train = Y_train[:, i]
    clf = LogisticRegression()
    try:
        proba, cross_val, cv_score = predict_one_class(X_train, X_test, y_train, clf, n_splits=3, silent=True)  
    except ValueError: ## ошибка недостаточного кол-ва примеров
        pass
    probas.append(proba)
    cross_vals.append(cross_val)
    cv_scores.append(cv_score)
print ()
print(f'TOTAL CV:{np.mean(cv_scores)}')


TOTAL CV:0.8938800060580677


In [30]:
Y = section_1_ohe.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(TF_X.toarray(), Y, test_size=0.2)

base_lr = LogisticRegression()
section_1_ovr = OneVsRestClassifier(base_lr)
section_1_ovr.fit(X_train, Y_train)
Y_pred_ovr = section_1_ovr.predict_proba(X_test)

section_1_ovr.fit(TF_X.toarray(), Y)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [31]:
Y = section_2_ohe.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(TF_X.toarray(), Y, test_size=0.1)

base_lr = LogisticRegression()
section_2_ovr = OneVsRestClassifier(base_lr)
section_2_ovr.fit(X_train, Y_train)
Y_pred_ovr = section_2_ovr.predict_proba(X_test)

section_2_ovr.fit(TF_X.toarray(), Y)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [32]:
Y = section_3_ohe.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(TF_X.toarray(), Y, test_size=0.1)

base_lr = LogisticRegression()
section_3_ovr = OneVsRestClassifier(base_lr)
section_3_ovr.fit(X_train, Y_train)
Y_pred_ovr = section_3_ovr.predict_proba(X_test)

section_3_ovr.fit(TF_X.toarray(), Y)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

## Dumping models

In [33]:
dump(section_1_ovr, 'section_1_ovr.joblib')
dump(section_2_ovr, 'section_2_ovr.joblib')
dump(section_3_ovr, 'section_3_ovr.joblib')

['section_3_ovr.joblib']

In [34]:
def predict_question_classes(text, vectorizer, encoders=[], models=[]):
    probable_classes = []
    clean_text = preprocess_text(text)
    X = [clean_text]
    vectorized = vectorizer.transform(X)
    for encoder, model in zip(encoders, models):
        prediction = model.predict_proba(vectorized)[0]
        categories = encoder.categories_[0]
        answer = sorted(zip(prediction, categories), reverse=True)[0]
        probable_classes.append(f'{answer[1]}: {answer[0]}')
    return probable_classes

In [35]:
predict_question_classes('are you able to copy/duplicate projects with subtasks?', vectorizer=vectorizer, 
                         encoders=[section_1_encoder, section_2_encoder, section_3_encoder], 
                         models=[section_1_ovr, section_2_ovr, section_3_ovr])

['Tasks, Folders, Projects and Spaces: 0.6007777298179908',
 'Advanced: 0.09398993974112084',
 'Subtasks: 0.011257968911922525']